In [30]:
import pandas as pd

bridges_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_processed.csv"  # DOMA
survey_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv"
# bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Bridges_processed.csv" # Práca

df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

In [31]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

df_bridges_all["Číslo_PK"] = df_bridges_all["Číslo_PK"].str.upper()



df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023'],
      

In [32]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges_SSC = df_bridges_all[
    ((df_bridges_all['Správca_úseku'] == "SSC - IVSC ZA") |
     (df_bridges_all['Správca_úseku'] == "SSC - IVSC KE") |
     (df_bridges_all['Správca_úseku'] == "SSC - IVSC BA")) &
    (df_bridges_all['n_2023'] > 0) &
    (df_bridges_all['ID_mosta'] != "Nezistený")
].copy()
filtered_bridges_SSC.to_excel('filtered_bridges_SSC.xlsx', index=False)
# Display the filtered DataFrame
filtered_bridges_SSC.shape

(1364, 51)

In [33]:
# Locate rows with "ID_mosta" equal to M2103 or M7498
rows_to_update = filtered_bridges_SSC[(filtered_bridges_SSC["ID_mosta"] == "M2103") | (
    filtered_bridges_SSC["ID_mosta"] == "M7498")]

# Update the "Číslo_PK" values for the located rows to "64"
filtered_bridges_SSC.loc[rows_to_update.index, "Číslo_PK"] = "64"

In [34]:
import pandas as pd

# Assuming df_survey is your DataFrame
filtered_df_survey = df_survey.copy()
# Step 1: Remove rows representing roads that start with letters
filtered_df_survey = filtered_df_survey[~filtered_df_survey['CESTA'].str.match(
    r'^[a-zA-Z]')]

# Step 2: Remove all letters from the remaining road names
filtered_df_survey['CESTA'] = filtered_df_survey['CESTA'].str.replace(
    r'\D', '')

# Step 3: Keep only the rows where the remaining name has exactly two characters
filtered_df_survey = filtered_df_survey[filtered_df_survey['CESTA'].str.len(
) <= 2]

# Display the filtered DataFrame
print(filtered_df_survey)

       ÚSEK CESTA      OKRES     T      O   M      S
32    80080     2    Malacky   561   4810  65   5436
33    80091     2    Malacky  1537  16148  87  17772
34    80092     2    Malacky   720   8683  66   9469
35    80093     2    Malacky   964   9990  87  11041
36    80098     2    Malacky   958   8331  89   9378
...     ...   ...        ...   ...    ...  ..    ...
2582  92891    78  Námestovo  1898  10646  23  12567
2583  92892    78  Námestovo  2100  13967  67  16134
2584  92896    78  Námestovo  1296   6359  42   7697
2585  92897    78  Námestovo   860   6737  28   7625
2586  92899    78  Námestovo   814   6941  26   7781

[926 rows x 7 columns]


In [35]:
# Get unique values of "Okres" column in filtered_bridges_SSC DataFrame
filtered_bridges_SSC_okres_values = set(
    filtered_bridges_SSC["Okres"].value_counts().keys())

# Get unique values of "OKRES" column in df_survey DataFrame
df_survey_okres_values = set(df_survey["OKRES"].value_counts().keys())

# Check if all values from filtered_bridges_SSC are in df_survey
is_subset = filtered_bridges_SSC_okres_values.issubset(df_survey_okres_values)

# Display the result
if is_subset:
    print("All values from bridges are present in survey.")
else:
    print("Not all values from bridges are present in survey.")

All values from bridges are present in survey.


In [36]:
# Get unique values of "Číslo_PK" column in filtered_bridges_SSC DataFrame
filtered_bridges_SSC_cesta_values = set(
    filtered_bridges_SSC["Číslo_PK"].value_counts().keys())

# Get unique values of "CESTA" column in df_survey DataFrame
df_survey_cesta_values = set(df_survey["CESTA"].value_counts().keys())

# Check if all values from filtered_bridges_SSC are in df_survey
is_subset = filtered_bridges_SSC_cesta_values.issubset(df_survey_cesta_values)

# Display the result
if is_subset:
    print("All values from bridges are present in survey.")
else:
    print("Not all values from bridges are present in survey.")

All values from bridges are present in survey.


In [37]:
# Calculate the mean traffic intensity for each road section (CESTA) and district (OKRES) and convert to integer
mean_traffic_intensity = df_survey.groupby(['CESTA', 'OKRES'])[
    'S'].mean().reset_index()
mean_traffic_intensity['S'] = mean_traffic_intensity['S'].astype(
    int)  # Convert mean values to integer

# Rename columns in mean_traffic_intensity for merging
mean_traffic_intensity.rename(
    columns={'CESTA': 'Číslo_PK', 'OKRES': 'Okres', 'S': 'inten'}, inplace=True)

# Merge the mean traffic intensity back to the filtered bridges dataframe on both road section and district
filtered_bridges_SSC = pd.merge(
    filtered_bridges_SSC, mean_traffic_intensity, how='left', on=['Číslo_PK', 'Okres'])

# Replace NaN values with 0 in the "inten" column
filtered_bridges_SSC["inten"] = filtered_bridges_SSC["inten"].fillna(0)

# Convert the "inten" column to integers
filtered_bridges_SSC["inten"] = filtered_bridges_SSC["inten"].astype(int)

In [38]:
# Filter bridges with inten == 0
bridges_with_zero_inten = filtered_bridges_SSC[filtered_bridges_SSC["inten"] == 0]

# Display the filtered bridges
bridges_with_zero_inten[["ID_mosta",
                         "Názov_mosta", "Číslo_PK", "Okres", "inten"]]

,ID_mosta,Názov_mosta,Číslo_PK,Okres,inten
88,M1892,15_019 Most cez bezmenný potok za obcou Duplín,15,Svidník,0
374,M899,09_109 Most cez bezmenný potok ori osade Jerichov,9,Partizánske,0
375,M3959,09_112 Most cez potok na hranici okresu Partiz...,9,Partizánske,0


TU JE NUTNÉ PREDEFINOVAŤ nejakú hodnotu, skúsiť nájsť z mapy ktorá nejde teraz lol

In [39]:
# Define conditions and corresponding values for "inten"
conditions_values = {
    "M1892": 6260,
    "M899": 4651,
    "M3959": 4651
}

# Update "inten" column based on the conditions
for condition, value in conditions_values.items():
    filtered_bridges_SSC.loc[filtered_bridges_SSC["ID_mosta"]
                             == condition, "inten"] = value

In [40]:
filtered_bridges_SSC.loc[:, 'F_y'] = filtered_bridges_SSC['Rok_postavenia'].apply(lambda x:
                                                                                  5 if x <= 1965
                                                                                  else (4 if 1965 < x <= 1975
                                                                                        else (3 if 1976 < x <= 1985
                                                                                              else (2 if 1986 < x <= 1996
                                                                                                    else 1))))

In [41]:
filtered_bridges_SSC['F_y'].value_counts()

F_y
5    722
4    206
1    179
3    172
2     85
Name: count, dtype: int64

In [42]:
# Create new column "F_c" and assign values from "n_2023"
filtered_bridges_SSC['F_c'] = filtered_bridges_SSC['n_2023']

In [43]:
filtered_bridges_SSC['F_c'].value_counts()

F_c
4    452
5    290
3    216
2    160
6    148
1     77
7     21
Name: count, dtype: int64

In [66]:
filtered_bridges_SSC['Material'].value_counts()

# Filter bridges where "Iná prekážka" is present in the list of "Predmet_premostenia"
filtered_bridges_with_nezis = filtered_bridges_SSC[filtered_bridges_SSC['Material'].apply(
    lambda x: 'Nezistený' in x)]

filtered_bridges_with_nezis[[
    "ID_mosta", "Okres", "Predmet_premostenia", "Dĺžka_premostenia", "inten"]]

,ID_mosta,Okres,Predmet_premostenia,Dĺžka_premostenia,inten
1050,M1767,Humenné,[Nezistený],5.380,17573
1251,M1776,Prešov,[Vodný tok],5.080,9986
1252,M4670,Prešov,[Vodný tok],5.080,9986
1324,M9578,Prešov,[Nezistený],2.300,9986
1332,M9672,Senec,[Cestná komunikácia],124.160,13136
1333,M9665,Prešov,[Vodný tok],161.974,18761
1334,M9666,Prešov,[Vodný tok],58.558,18761
1335,M9668,Prešov,[Vodný tok],3.000,18761
1336,M9669,Prešov,[Vodný tok],3.000,18761


In [44]:
# Define a dictionary mapping materials to their corresponding values
material_mapping = {
    'prefabrikovaný predpätý betón': 10,
    'monolitický predpätý betón': 8,
    'monolitický železobetón': 6,
    'prefabrikovaný železobetón': 6,
    'monolitický prostý betón': 6,
    'spriahnutá betón-betón': 5,
    'spriahnutá oceľ-betón': 5,
    'oceľ -  plnostenná': 5,
    'oceľ -  priehradová': 5,
    'oceľobetón': 5,
    'tehla': 2,
    'kameň': 2,
    'Nezistený': 6,
    'iný materiál': 6
}

# Create a new column [F_f] based on the mapped values
filtered_bridges_SSC['F_f'] = filtered_bridges_SSC['Material'].map(
    material_mapping)

In [45]:
filtered_bridges_SSC['F_f'].value_counts()

F_f
6     906
10    319
8      63
5      54
2      22
Name: count, dtype: int64

In [46]:
# Define a dictionary mapping 'Druh_konštrukcie' values to their corresponding values
construction_mapping = {
    'dosková': 0,
    'trámová': 2,
    'klenbová': 0,
    'rámová': 0,
    'oblúková': 0,
    'Nezistený': 0,
    'iná': 0,
    'priehradová': 0,
    'zavesená': 0
}

# Apply condition to update 'F_f' only where 'Material' is 'prefabrikovaný predpätý betón'
filtered_bridges_SSC.loc[filtered_bridges_SSC['Material'] == 'prefabrikovaný predpätý betón',
                         'F_f'] += filtered_bridges_SSC['Druh_konštrukcie'].map(construction_mapping)

In [47]:
filtered_bridges_SSC['F_f'].value_counts()

F_f
6     906
12    170
10    149
8      63
5      54
2      22
Name: count, dtype: int64

In [48]:
# Split the values in the "Predmet_premostenia" column into lists
filtered_bridges_SSC["Predmet_premostenia"] = filtered_bridges_SSC["Predmet_premostenia"].str.split(
    ",  ")

In [64]:
# Filter bridges where "Iná prekážka" is present in the list of "Predmet_premostenia"
filtered_bridges_with_ina_prekazka = filtered_bridges_SSC[filtered_bridges_SSC['Predmet_premostenia'].apply(
    lambda x: 'Nezistený' in x)]

filtered_bridges_with_ina_prekazka[[
    "ID_mosta", "Okres", "Predmet_premostenia", "Dĺžka_premostenia", "inten"]]

,ID_mosta,Okres,Predmet_premostenia,Dĺžka_premostenia,inten
10,M6131,Malacky,[Nezistený],4.55,10359
320,M2967,Žilina,[Nezistený],123.50,33554
321,M6758,Žilina,[Nezistený],123.50,33554
684,M5083,Galanta,[Nezistený],4.00,9543
761,M6764,Prievidza,[Nezistený],10.35,12089
1000,M4488,Svidník,[Nezistený],4.00,7063
1049,M2524,Humenné,[Nezistený],2.52,17573
1050,M1767,Humenné,[Nezistený],5.38,17573
1257,M3830,Žilina,[Nezistený],67.35,25447
1323,M9571,Košice - okolie,[Nezistený],2.50,11037


M5852   F_Q = 5 -> estakáda
M2120   F_Q = 4 -> estakáda
M410    F_Q = 1 -> Nič
M6696   F_Q = 1 -> Nevidím nič
M4932   F_Q = 1 -> Nevidím nič
M3423   F_Q = 1 -> Nevidím nič
M1155   F_Q = 1 -> Nevidím nič
M1140   F_Q = 1 -> Nevidím nič
M1470   F_Q = 1 -> Kanál
M9678   F_Q = 3 -> Splavný tok
M6131   F_Q = 2 -> Vodný tok
M2967   F_Q = 5 -> Železnica
M6758   F_Q = 3 -> Nejaká cesta
M5083   F_Q = 1 -> Pole
M6764   F_Q = 3 -> Podchod
M4488   F_Q = 1 -> Kanál
M2524   F_Q = 1 -> pole
M1767   F_Q = 1 -> Suchý járok
M3830   F_Q = 3 -> Cesty
M9571   F_Q = 1 -> Potok
M9578   F_Q = 1 -> Potok, 48.836339, 21.285492 asi
M9561   F_Q = 3 -> Cesta, 49.282795, 21.270006
M9565   F_Q = 3 -> Cesta, 49.283327, 21.273450

In [49]:
# Define the classification key as a dictionary
classification_key = {
    "Vodný tok": 2,
    "Železnica": 5,
    "Kanál": 1,
    "Cestná komunikácia": 4,
    "Miestna komunikácia": 3,
    "Účelová komunikácia": 2,
    "Záplavové územie": 2,
    "Nezistený": 3,
    "Iná prekážka": 1,
    "Podchod pre chodcov": 3,
    "Údolie": 2,
    "Vodná nádrž": 3
}

# Mapping function to get the highest value for each item


def get_highest_value(item):
    # Evaluate the list from string representation if necessary
    # Remove the brackets and split by comma if item is a string representation of a list
    if isinstance(item, str):
        item = item.strip("[]").split(", ")
    # Replace category names in the list with their corresponding values
    values = [classification_key.get(
        category.strip("[]'\""), 0) for category in item]
    # Return the highest value found
    return max(values)


# Apply the mapping function to each item in the column and create a new column ["F_q"]
filtered_bridges_SSC["F_q"] = filtered_bridges_SSC["Predmet_premostenia"].apply(
    get_highest_value)

In [50]:
filtered_bridges_SSC["F_q"].value_counts()

F_q
1    998
2    131
5    107
3     72
4     56
Name: count, dtype: int64

In [51]:
import pandas as pd

# Assuming filtered_bridges_SSC is your DataFrame

# Select columns starting with "n_"
condition_columns = filtered_bridges_SSC.filter(like='n_').columns

# Concatenate values of selected columns into a new column "STS"
filtered_bridges_SSC['STS'] = filtered_bridges_SSC[condition_columns].apply(
    lambda x: ','.join(x.dropna().astype(str)), axis=1)

# Split the "STS" column into lists of integers
filtered_bridges_SSC['STS_List'] = filtered_bridges_SSC['STS'].apply(
    lambda x: [int(n) for n in x.split(',')])

In [52]:
def filter_series(y):
    # Convert the series to a NumPy array for easier manipulation
    y = np.array(y)

    # Remove leading zeros (if any)
    y_non_zero = y[y > 0]

    # If after removing zeros the series is empty or consists of a constant sequence, return it as is
    if len(y_non_zero) <= 1 or np.all(y_non_zero == y_non_zero[0]):
        return y_non_zero.tolist()

    # Now, find the first change in the non-zero part of the series
    first_change_index = np.where(y_non_zero[:-1] != y_non_zero[1:])[0][0] + 1

    # The filtered series should start from the first non-zero value to the end of changes
    # Adjust to include the first non-zero value that changes
    y_filtered = y_non_zero[first_change_index - 1:]

    return y_filtered.tolist()


# Example usage:
# Assuming 'filtered_bridges_SSC' is your DataFrame and 'STS_List' contains the lists of integers
filtered_bridges_SSC['STS_filt'] = filtered_bridges_SSC['STS_List'].apply(
    filter_series)

NameError: name 'np' is not defined

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress


def compute_slope_filtered(y):
    # Check if the list is empty or has insufficient data points
    if not y or len(y) <= 1:
        return 0

    # Convert the series to a NumPy array for easier manipulation
    y = np.array(y)

    # Use the entire filtered series directly
    y_filtered = y

    # Create x indices for the filtered series, starting from 0
    x_filtered = np.arange(len(y_filtered))

    # Compute the slope using linear regression
    slope, intercept, r_value, p_value, std_err = linregress(
        x_filtered, y_filtered)

    # Calculate the scaling factor as the ratio of the length of filtered values to 12 (the total number of expected data points)
    scaling_factor = len(y_filtered) / 12
    scaled_slope = slope * scaling_factor

    return scaled_slope


# Apply the updated function to the 'STS_filt' column to compute the scaled degradation rate
filtered_bridges_SSC['Scaled_Degradation_Rate'] = filtered_bridges_SSC['STS_filt'].apply(
    compute_slope_filtered)

# Optionally, view or analyze the distribution of scaled degradation rates
print(filtered_bridges_SSC['Scaled_Degradation_Rate'].describe())

count    1364.000000
mean        0.059033
std         0.118296
min        -0.575000
25%         0.000000
50%         0.055556
75%         0.141667
max         0.500000
Name: Scaled_Degradation_Rate, dtype: float64


In [ ]:
# Sort the DataFrame by the maximum value of Degradation_Rate
sorted_df = filtered_bridges_SSC.sort_values(
    by='Scaled_Degradation_Rate', ascending=False)

sorted_df[["ID_mosta", "STS", "STS_filt", "Scaled_Degradation_Rate"]].head(5)

,ID_mosta,STS,STS_filt,Scaled_Degradation_Rate
415,M1155,"2,2,2,2,2,2,2,2,2,2,2,5","[2, 5]",0.5
676,M1963,"3,3,3,3,3,3,3,3,3,3,3,6","[3, 6]",0.5
1043,M7192,"3,3,3,3,3,3,3,3,3,3,3,6","[3, 6]",0.5
1690,M6287,"1,1,1,1,1,1,1,5,5,5,6,6","[1, 5, 5, 5, 6, 6]",0.4
1370,M4413,"2,2,2,2,2,2,2,5,5,5,6,7","[2, 5, 5, 5, 6, 7]",0.4


In [ ]:
def assign_fd(m):
    if m < 0:
        return 0
    elif 0.0 <= m < 0.1:
        return 0
    elif 0.1 <= m < 0.2:
        return 1
    elif 0.2 <= m < 0.4:
        return 2
    elif m >= 0.4:
        return 3


# Apply the assign_fd function to the 'Scaled_Degradation_Rate' column to compute 'F_d'
filtered_bridges_SSC['F_d'] = filtered_bridges_SSC['Scaled_Degradation_Rate'].apply(
    assign_fd)

In [ ]:
# Computing the new column R using the given formula
filtered_bridges_SSC['R'] = ((((1 * filtered_bridges_SSC['F_y'] + filtered_bridges_SSC['F_f'] +
                                4*(filtered_bridges_SSC['F_d'] + filtered_bridges_SSC['F_c'])) * 0.5 *
                               filtered_bridges_SSC['F_q']) - 0) / 228) * 100

# # Computing the new column R using the given formula
# filtered_bridges_SSC['R'] = (1 * filtered_bridges_SSC['F_y'] + filtered_bridges_SSC['F_f'] +
#                                 4*(filtered_bridges_SSC['F_d'] + filtered_bridges_SSC['F_c']) +
#                                filtered_bridges_SSC['F_q'])

In [ ]:
filtered_bridges_SSC_sorted_asc = filtered_bridges_SSC.sort_values(
    by='R', ascending=False)

In [ ]:
filtered_bridges_SSC_sorted_asc[["ID_mosta", "Názov_mosta",
                                 "Material", "Dĺžka_premostenia", "Druh_konštrukcie", "F_c", "F_d", "F_y", "F_f",   "F_q", "R"]].head(60)

,ID_mosta,Názov_mosta,Material,Dĺžka_premostenia,Druh_konštrukcie,F_c,F_d,F_y,F_f,F_q,R
1807,M6028,79_021 Most cez železničnú trať a miestnu komu...,prefabrikovaný predpätý betón,91.76,trámová,7,2,2,12,5,54.824561
293,M4815,"18_390 Most cez železničnú trať, k.ú. Poprad",prefabrikovaný predpätý betón,116.80,dosková,7,2,4,10,5,54.824561
1694,M1218,77_012 Most nad železničnou traťou za obcou Po...,monolitický železobetón,8.20,klenbová,7,2,5,6,5,51.535088
954,M2437,62_013 Nadjazd nad žel. traťou Galanta-Leopold...,prefabrikovaný predpätý betón,39.75,dosková,7,1,5,10,5,51.535088
972,M3483,"63_023B Most nad železnicou a MK, Estakáda v K...",prefabrikovaný predpätý betón,240.28,trámová,6,1,3,12,5,47.149123
971,M1368,"63_023A Most nad železnicou a MK, Estakáda v K...",prefabrikovaný predpätý betón,232.11,trámová,6,1,3,12,5,47.149123
670,M2448,51_087 Most nad železničnou traťou pred obcou ...,prefabrikovaný predpätý betón,33.97,dosková,6,2,1,10,5,47.149123
912,M3992,61_068 Most cez odstránenú železničnú vlečku z...,monolitický železobetón,7.70,dosková,6,2,5,6,5,47.149123
943,M1556,"62_002 Most nad železnicou, k.ú. Senec",prefabrikovaný predpätý betón,47.00,dosková,5,2,4,10,5,46.052632
655,M6701,51_062 Most nad železnicou v obci Holíč,prefabrikovaný predpätý betón,93.00,trámová,5,1,4,12,5,43.859649


In [ ]:
# Filter bridges where 'F_q' is equal to 5
filtered_bridges_F_q_5 = filtered_bridges_SSC[filtered_bridges_SSC['F_q'] == 5]

filtered_bridges_F_q_5.shape

(107, 61)

In [ ]:
filtered_bridges_SSC_sorted_asc = filtered_bridges_SSC.sort_values(
    by='R', ascending=False)

filtered_bridges_SSC_sorted_asc['R'].max()

54.824561403508774